In [ ]:
%matplotlib inline

# MNI Atlas Reader

A Python interface for generating coordinate tables and region labels from fMRI statistical images.

In [ ]:
import os
import matplotlib.pyplot as plt
from nilearn.datasets import fetch_neurovault_motor_task, fetch_neurovault_auditory_computation_task
from nilearn import plotting
import atlasreader

## Example Stat Map

First, let's grab an example stat map. We'll use `nilearn` to grab a motor task stat map from [neurovault](https://neurovault.org/). Note that you'll need the most up to date version of `nilearn` in order to run `fetch_neurovault_motor_task()`.

In [ ]:
motor_images = fetch_neurovault_motor_task()
stat_img = motor_images.images[0]

In [ ]:
# make an output folder in the directory of stat_img
outpath = os.path.join(os.path.dirname(stat_img), 'clusters')

In [ ]:
plt.figure(figsize=(12, 10))
plotting.plot_stat_map(stat_img, threshold=3, black_bg=True, vmax=10,
                       title='Finger tapping task', cut_coords=[60, -19, 46])
plt.show()

## Atlas Reader from the Command Line

Now that we have our stat map, we can run `atlasreader` directly from the command line to extract out the peaks and cluster information. Let's begin with running the `help` argument so we can look at the input parameters.

In [ ]:
%%bash 
atlasreader -h

Great, so now we can run `atlasreader` and provide our stat map and the out directory that we made earlier. Let's threshold our _t_ values to 5 and set the `extent` to 10 in order to get robust clusters of at least 10 or more voxels. We'll also set our probability threshold to 33% to remove small probabilities. 

In [ ]:
%%bash -s "$stat_img" "$outpath"
atlasreader $1 -a Harvard_Oxford -t 5 -c 10 -p 33 -o $2

Let's check out peak and cluster information, respectively:

In [ ]:
# read in main output file
import pandas as pd

df = pd.read_csv(os.path.join(outpath, 'image_10426_peaks.csv'))
df.head(20)

In [ ]:
df = pd.read_csv(os.path.join(outpath, 'image_10426_clusters.csv'))
df.head(20)

We can also grab the cluster images that were created from our output directory. Note that the crosshair is placed on the peak voxel within the cluster:

In [ ]:
from IPython.display import Image
Image(filename=os.path.join(outpath, 'image_10426_cluster01.png'), width=750)

In [ ]:
from IPython.display import Image
Image(filename=os.path.join(outpath, 'image_10426_cluster02.png'), width=750)

## Calling from Python

You can also import the `create_output` function from the `atlasreader` module itself, and run it directly from python. Note that in this example, another atlas (`aal`) is added. We'll repeat what we ran earlier.

In [ ]:
from atlasreader.atlasreader import create_output

create_output(stat_img, ['Harvard_Oxford', 'aal'],
              voxel_thresh=3, cluster_extent=10,
              prob_thresh=33, outdir=outpath)

In [ ]:
df = pd.read_csv(os.path.join(outpath, 'image_10426_peaks.csv'))
df.head(20)

In [ ]:
df = pd.read_csv(os.path.join(outpath, 'image_10426_clusters.csv'))
df.head(20)